# deeptrack.image

<a href="https://colab.research.google.com/github/DeepTrackAI/DeepTrack2/blob/develop/tutorials/3-advanced-topics/DTAT311_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install deeptrack  # Uncomment if running on Colab/Kaggle.

This advanced tutorial introduces the module deeptrack.image.

In [2]:
import numpy as np

## 1. What is an Image?

An `Image` is a container used by a feature to store both the generated image and the properties used to generate it. It is a subclass of numpy `ndarray`, so that any operation that works for `ndarrays` will also work for `Image`. `Image` has also a field `properties`, which contains the information about how the image has been generated (see also [properties_example](properties_example.ipynb)). Specifically, `properties` is a list of dictionaries, where each dictionary holds the current values of the properties of a feature (with the names of the properties as keys). This list is ordered as the features have been resolved.

By storing the properties used to resolve the image, information about the image can be accessed without access to the feature series. This allows features to change their behaviour depending on what is already in the image. This also allows to extract numeric information about the image (e.g., to be used in supervised learning).

**NOTE:** By default, feartures return numerical arrays. To return objects of the `Image` class, you need to call the `.store_properties()` method on the individual feature.

In [3]:
from deeptrack.features import Feature
from deeptrack.image import Image

class Particle(Feature):
    def get(self, image, position=None, **kwargs):
        # Code for simulating a particle not included
        return image
    
image_shape = (256, 256)
    
particle = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
)
particle.store_properties()  # Return Image instead of NumPy array.

input_image = Image(np.zeros(image_shape))
# input_image = np.zeros(image_shape)  # The input image can be a NumPy array.

output_image = particle.resolve(input_image)

print(output_image)
print(output_image.properties)

Image(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]))
[{'position': array([29.1770169, 61.4817317]), 'name': 'Particle'}]


You can use the method `.merge_properties_from()` to merge the properties of two images.
If the two images are intetical (they are resolved from the same feature without an update), only one set of properties is kept.

In [4]:
# The same particle is resolved again.
second_identical_image = particle.resolve(input_image)

output_image.merge_properties_from(second_identical_image)

print(output_image.properties)

[{'position': array([29.1770169, 61.4817317]), 'name': 'Particle'}]


However, if you update the feature between merges, the properties are treated as distinct. This is true even if the properties are not randomized.

In [5]:
particle.update()
updated_image = particle.resolve(input_image)

output_image.merge_properties_from(updated_image)

print(output_image.properties)

[{'position': array([29.1770169, 61.4817317]), 'name': 'Particle'}, {'position': array([ 52.69698425, 147.84653731]), 'name': 'Particle'}]


## 2. Extracting the Property Values

To retrieve information from the image, you can iterate through the list and extract the needed properties. The function `.get_property()` is a shorthand for this. It retrieves a property or a list of properties from an image. By default it retrieves a single instance of a property (the first instance that it finds).

In [6]:
output_image.get_property("name")

'Particle'

In [7]:
output_image.get_property("position")

array([29.1770169, 61.4817317])

To retrieve all instances of a property, set `get_one=False`.

In [8]:
output_image.get_property("name", get_one=False)

['Particle', 'Particle']

In [9]:
output_image.get_property("position", get_one=False)

[array([29.1770169, 61.4817317]), array([ 52.69698425, 147.84653731])]

Finally, if the property is not found, `default` is returned instead (by default `default=None`). 

In [10]:
output_image.get_property("a_property_that_does_not_exist",
                          default="Not found!")

'Not found!'

## 3. Storing Metadata about an Image Not Used by the Feature

Sometimes it is convenient to store some information about a feature beyond what is strictly necessary to generate it. This can be done by passing additional keyword arguments to the constructor of a feature. These will be stored as properties just like any other input to the constructor.

For example, you may want to identify one particle to track.

In [11]:
particle_to_track = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=True,
)
particle_to_track.store_properties()  # Return Image.

particle_not_to_track = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    track_me=False,
)
particle_not_to_track.store_properties()  # Return Image.

input_image = Image(np.zeros(image_shape))

# 5 particles are resolved, 4 of which are not tracked.
particles = (particle_not_to_track ^ 4) >> particle_to_track
particles.store_properties()  # Return Image.

output_image = particles.resolve(input_image)

print(output_image.get_property("track_me", get_one=False))

[False, False, False, False, True]


Or you might want to store the randomized diffusion constant of a particle used to generate physically-accurate sequences of images.

In [12]:
particle = Particle(
    position=lambda: np.random.rand(2) * np.array(image_shape),
    diffusion=lambda: 1 + np.random.rand() * 4,
) 
particle.store_properties()  # Return Image.

input_image = Image(np.zeros(image_shape))

# 3 particles are resolved, and their diffusion is stored.
particles = particle ^ 3
particles.store_properties()  # Return Image.

output_image = particles.resolve(input_image)

print(output_image.get_property("diffusion", get_one=False))

[3.011554224581562, 1.3630043223595303, 4.588298571181895]
